In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from random import uniform

#### Clases para el Adaline y Capa que contiene varias neuronas

In [2]:
class Adaline:

    def __init__(self, n_entradas, bias=None, alfa=0.1):
        self.pesos_sinapticos = np.random.uniform(
            low=-0.05, high=0.05, size=(n_entradas,))
        self.n_entradas = n_entradas
        self.bias = bias if bias else uniform(0,0.1)
        self.alfa = alfa

    def calcular(self, estimulo):
        y = np.dot(self.pesos_sinapticos, estimulo)
        y += self.bias
        self.y = self.lineal(y)
        return self.y

    def entrenar_estocastico(self, estimulo, resultado_esperado):
        """
            LMS Estocastico
        """
        self.calcular(estimulo)
        error = (resultado_esperado - self.y)
        delta = self.alfa * error * estimulo
        
        # Actualizacion
        self.pesos_sinapticos = self.pesos_sinapticos + delta
        self.bias = self.bias + error * self.alfa
        return self.pesos_sinapticos

    def entrenar_batch(self, estimulos, resultados_esperados):
        """
            LMS Batch
        """
        delta = np.zeros(self.n_entradas)
        for estimulo, respuesta in zip(estimulos, resultados_esperados):
            self.calcular(estimulo)
            error = (respuesta - self.y)
            delta += error * estimulo
            self.bias += error * self.alfa
        self.pesos_sinapticos = self.pesos_sinapticos + self.alfa * delta

    # Por seguir con el algoritmo
    def lineal(self, a):
        """
            Funcion de transferencia lineal
        """
        return a

    def describir(self):
        print("PESOS SINAPTICOS: ", self.pesos_sinapticos)
        print("RESPUESTA: ", self.y)


In [3]:
class Capa:

    def __init__(self, n_adaline, n_entradas, alfa=0.1):
        self.n_adaline = n_adaline
        self.n_entradas = n_entradas
        self.adalines = [Adaline(n_entradas, alfa=alfa)
                             for i in range(n_adaline)]

    def entrenar(self, entrada, respuesta):
        """
            Entrenamiento estocastico de las neuronas
        """
        for i in range(self.n_adaline):
            if respuesta == i:
                self.adalines[i].entrenar_estocastico(entrada, 1)
            else:
                self.adalines[i].entrenar_estocastico(entrada, 0)

    def calcular(self, estimulo):
        """
            Calcular el resultado dado un estimulo a las neuronas
        """
        respuesta = []
        for i in range(self.n_adaline):
            respuesta.append(self.adalines[i].calcular(estimulo))

        return respuesta

#### Dos funciones utiles para entrenar una capa y para verificar esa misma capa

In [4]:
def entrenar(capa, epocas=1):
    """
    Entrenar las neuronas de una capa
    """
    with open('../mnist_train.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',',
                                quoting=csv.QUOTE_NONNUMERIC)
        start = time.time()
        leido = 0
        for _ in range(epocas):
            leido = 0
            for row in csv_reader:
                # Dividimos los datos por 255 para tenerlos 
                # en un rango entre 0 y 1
                datos = np.array(row[1:]) / 255
                capa.entrenar(datos, row[0])
                leido += 1
                
            # Devolvemos el apuntador del archivo al inicio
            csv_file.seek(0)
        print(f"Tiempo: {time.time() - start}seg")
        print(f"No de datos usados para el entrenamiento: {leido}")

In [5]:
def verificar(capa):
    """
    Verificar si las neuronas de una capa dada estan entrenadas
    """
    with open('../mnist_test.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',',
                                quoting=csv.QUOTE_NONNUMERIC)
        line_count = 0
        resultados_buenos = 0
        resultados_malos = 0
        for row in csv_reader:
            resultado = row[0]
            datos = np.array(row[1:])
            res = capa.calcular(datos)
            rep = [0,res[0]]
            for i, j in enumerate(res):
                if j > rep[1] :
                    rep[0] = i
                    rep[1] = j

            if rep[0] == resultado:
                resultados_buenos += 1
            else:
                resultados_malos += 1
            line_count += 1

        
        print(f"Datos leido: {line_count}")
        print(
            f"Porcentaje de resultados buenos: {resultados_buenos / line_count * 100}%")
        print(
            f"Porcentaje de resultados malos: {resultados_malos / line_count * 100}%")

Realizamos los entrenamientos con las siguientes tasas de aprendizaje:
* 0.1
* 0.01
* 0.001

### Usando etha 0.1 con 10 epocas

In [15]:
c = Capa(10,784,0.1)

In [17]:
entrenar(c,10)

Tiempo: 274.76490235328674seg
No de datos usados para el entrenamiento: 60000


In [18]:
verificar(c)

Datos leido: 10000
Porcentaje de resultados buenos: 9.8%
Porcentaje de resultados malos: 90.2%


### Usando etha 0.01 con 10 epocas

In [9]:
c1 = Capa(10,784,0.01)

In [10]:
entrenar(c1,10)

Tiempo: 267.24732303619385seg
No de datos usados para el entrenamiento: 60000


In [11]:
verificar(c1)

Datos leido: 10000
Porcentaje de resultados buenos: 59.419999999999995%
Porcentaje de resultados malos: 40.58%


### Usando etha 0.001 con 10 epocas

In [12]:
c2 = Capa(10,784,0.001)

In [13]:
entrenar(c2, 10)

Tiempo: 292.37622570991516seg
No de datos usados para el entrenamiento: 60000


In [14]:
verificar(c2)

Datos leido: 10000
Porcentaje de resultados buenos: 74.96000000000001%
Porcentaje de resultados malos: 25.040000000000003%


### Extra usando etha 0.0001 con 10 epocas

In [19]:
c3 = Capa(10,784,0.0001)

In [20]:
entrenar(c3, 10)

Tiempo: 295.4783835411072seg
No de datos usados para el entrenamiento: 60000


In [21]:
verificar(c3)

Datos leido: 10000
Porcentaje de resultados buenos: 80.89%
Porcentaje de resultados malos: 19.11%


### Comparaciones con el perceptron

![](./resultado-perceptron.png)

Con lo que nos queda:

| Neurona | 0.1 | 0.01 | 0.001 |
| --- | --- | --- | --- | 
| Perceptron | 75.18% | 81.42% | 81.02% |
| Adaline | 9.8% | 59.41% | 74.96% |


### Conclusiones

Al parecer si ejecutamos la Capa de Adaline con Etha = 0.1 es una muy mala opcion 
Ahora, si vamos bajando esa tasa de aprendizaje vamos obteniendo mejores resultados
al final el mejor resultado fue con 0.0001 dando un porcentaje de resultados buenos 
de mas del 80%, sin embargo, con este valor no fue probado el perceptron.

Tendriamos que volver a ejecutar todos los valores de etha con mas epocas para 
tener mas informacion del comportamiento con esta neurona y con los datos de entrenamiento
sobre los que estamos trabajando.